In [9]:
qpath = r'../../../' # path to quetzal here
data = r'inputs/'
import sys
sys.path.append(qpath + r'quetzal/')

# import class
from quetzal.io.gtfs_reader import importer

### Read GTFS

In [10]:
feed = importer.GtfsImporter(path=data + r'madrid.zip', dist_units='m')
feed.clean()
feed.describe()

,indicator,value
0,agencies,[Empresa Municipal de Transportes]
1,running_services,"[FE, LJ, LA, VV, SA]"
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20201231
5,num_routes,215
6,num_trips,79196
7,num_stops,4724
8,num_shapes,426
9,num_frequencies,78520


Frequency conversion currently work only for one specific service and date, and on one given time period.  
It computes the average headway over this time period.

### Restrict to one date and merge services

In [11]:
feed = feed.restrict(dates=['20191210'])
feed.group_services()

In [12]:
feed.describe()

,indicator,value
0,agencies,[Empresa Municipal de Transportes]
1,running_services,[LA]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20191210
5,num_routes,210
6,num_trips,35867
7,num_stops,4715
8,num_shapes,420
9,num_frequencies,35647


In [13]:
feed.map_trips(feed.trips.groupby('route_id').first().trip_id.head(100))

### Build simplified patterns by clustering stops

In [14]:
feed.build_stop_clusters(distance_threshold=300)

In [15]:
feed.build_patterns(on='cluster_id')

In [16]:
feed.describe()

,indicator,value
0,agencies,[Empresa Municipal de Transportes]
1,running_services,[LA]
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20191210
5,num_routes,210
6,num_trips,35867
7,num_stops,4715
8,num_shapes,420
9,num_frequencies,35647


### Convert to frequencies

In [ ]:
time_range = ['08:00:00', '10:00:00']  # time format must be HH:MM:SS
feed_f = feed.convert_to_frequencies(time_range=time_range)

 38%|██████████████████████████████▎                                                 | 138/364 [00:31<01:20,  2.81it/s]

In [ ]:
feed_f.describe()

### Build nodes and links

In [ ]:
feed_f.build_links_and_nodes()

In [ ]:
feed_f.nodes.head(1).T

In [ ]:
feed_f.links.head(1).T

In [ ]:
feed_f.links.sum()

In [ ]:
imp.links.sum()

## All in one line
While we recommand to build the nodes and links gradually by exploring the GTFS data first, it is also possible to do it in one line.

In [2]:
feed = importer.GtfsImporter(path=data + r'madrid.zip', dist_units='m')
feed.describe()

,indicator,value
0,agencies,[Empresa Municipal de Transportes]
1,running_services,"[FE, LJ, LA, VV, SA]"
2,timezone,Europe/Madrid
3,start_date,20191210
4,end_date,20201231
5,num_routes,215
6,num_trips,79196
7,num_stops,4724
8,num_shapes,426
9,num_frequencies,78520


In [3]:
imp = feed.build(
    date='20191210',
    time_range=['08:00:00', '10:00:00'],
    cluster_distance_threshold=300 # by default: None and no clustering
)

Restricting to date…
Grouping services…
Cleaning…
Clustering nodes…
Building patterns…
Converting to frequencies…


100%|████████████████████████████████████████████████████████████████████████████████| 364/364 [01:11<00:00,  5.09it/s]


Building links and nodes…


In [4]:
imp.links.head()

,index,a,trip_id,link_sequence,departure_time,b,arrival_time,time,headway,pattern_id,...,shape_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,geometry
0,0,273,001_0,1,28980.0,716,29049.0,69.0,143.0,001_0,...,001_B,EMT,1,Plaza de Cristo Rey - Prosperidad,NaN,3,http://www.emtmadrid.es/aplicaciones/Itinerari...,0178BC,FFFFFF,"LINESTRING (442765.171 4477229.698, 442686.114..."
1,1,716,001_0,2,29049.0,718,29200.0,151.0,143.0,001_0,...,001_B,EMT,1,Plaza de Cristo Rey - Prosperidad,NaN,3,http://www.emtmadrid.es/aplicaciones/Itinerari...,0178BC,FFFFFF,"LINESTRING (442686.114 4477094.875, 442856.656..."
2,2,718,001_0,3,29200.0,722,29346.0,146.0,143.0,001_0,...,001_B,EMT,1,Plaza de Cristo Rey - Prosperidad,NaN,3,http://www.emtmadrid.es/aplicaciones/Itinerari...,0178BC,FFFFFF,"LINESTRING (442856.656 4476770.543, 443081.051..."
3,3,722,001_0,4,29346.0,2307,29515.0,169.0,143.0,001_0,...,001_B,EMT,1,Plaza de Cristo Rey - Prosperidad,NaN,3,http://www.emtmadrid.es/aplicaciones/Itinerari...,0178BC,FFFFFF,"LINESTRING (443081.051 4476499.090, 443125.817..."
4,4,2307,001_0,5,29515.0,725,29622.0,107.0,143.0,001_0,...,001_B,EMT,1,Plaza de Cristo Rey - Prosperidad,NaN,3,http://www.emtmadrid.es/aplicaciones/Itinerari...,0178BC,FFFFFF,"LINESTRING (443125.817 4476139.089, 443086.878..."


In [5]:
imp.nodes.head()

,stop_id,stop_code,stop_name,stop_desc,zone_id,stop_url,location_type,parent_station,cluster_id,geometry
0,1,1,Avenida Valdemarín - Blanca De Castilla,NaN,NaN,http://www.emtmadrid.es/aplicaciones/Estimacio...,NaN,NaN,1361,POINT (433633.776 4480229.030)
1,2,2,Avenida Valdemarín - La Salle,NaN,NaN,http://www.emtmadrid.es/aplicaciones/Estimacio...,NaN,NaN,1037,POINT (433371.223 4480068.179)
2,3,3,Blanca De Castilla - Camino De La Zarzuela,NaN,NaN,http://www.emtmadrid.es/aplicaciones/Estimacio...,NaN,NaN,70,POINT (433545.593 4479654.783)
3,4,4,Pléyades - Ana Teresa,NaN,NaN,http://www.emtmadrid.es/aplicaciones/Estimacio...,NaN,NaN,987,POINT (433521.776 4479264.241)
4,5,5,Pléyades - Osa Mayor,NaN,NaN,http://www.emtmadrid.es/aplicaciones/Estimacio...,NaN,NaN,1452,POINT (433503.695 4479042.383)


In [8]:
imp.links[['route_short_name', 'headway']].drop_duplicates()

,route_short_name,headway
0,1,143.0
29,1,185.0
57,2,57.0
87,2,67.0
115,3,80.0
149,3,69.0
176,4,99.0
202,4,106.0
227,5,128.0
254,5,142.0
